In [1]:
modello_per_analisi = "Opel_Corsa"
marca, modello = [p.lower() for p in modello_per_analisi.split("_")]
prezzo_minimo = 3000
prezzo_massimo = 30000
km_minimi = 0
km_massimi = 0
immatricolazione_minima = 2020
immatricolazione_massima = 2025
#cambio= "manuale"   ---   # automatico, semiautomatico, manuale     ----     ### solo per automobili.it si possono uinserire tutti i cambi separati da una virgola
#cambio non ancora implementato nelle funzioni di Url_builders

### 0. Import funzioni e librerie

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import time

In [3]:
#Scraping
from Utils.Scraping_Functions import autoscout_scraper, autosupermarket_scraper, automobile_it_scraper, subito_scraper
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import random

In [4]:

from Utils.Url_builders import build_automobile_url, build_subito_url, build_autosupermarket_url, build_autotorino_url, bild_autoscout_urls

### 1. Creazione e importazione URLs

In [7]:
# ------------------- USO DELLE FUNZIONI: Urls_building -------------------
url_asm = build_autosupermarket_url(marca, modello, prezzo_minimo, prezzo_massimo,
                                    km_minimi, km_massimi, immatricolazione_minima, immatricolazione_massima)

url_at = build_automobile_url(marca, modello, prezzo_minimo, prezzo_massimo,
                              km_minimi, km_massimi, immatricolazione_minima, immatricolazione_massima)

url_su = build_subito_url(marca, modello, prezzo_minimo, prezzo_massimo,
                              km_minimi, km_massimi, immatricolazione_minima, immatricolazione_massima)

url_ar = build_autotorino_url(marca_sito = 818, #codice di autotorino per Opel
                              modello_sito = 1028, #codice di autotorino per Corsa
                              prezzo_minimo=prezzo_minimo,prezzo_massimo=prezzo_massimo,km_minimi=km_minimi,km_massimi=km_massimi,
                              anno_min=immatricolazione_minima,anno_max=immatricolazione_massima)

urls_as = bild_autoscout_urls (marca, modello, prezzo_minimo, prezzo_massimo,
                              km_minimi, km_massimi, immatricolazione_minima, immatricolazione_massima)

# ------------------- STAMPA DEGLI URL -------------------
print("Automobile.it URL:", url_at)
print("Autosupermarket URL:", url_asm)
print("Subito.it URL:", url_su)
print("Autotorino URL:", url_ar)
print("Autoscout URLs:", urls_as)

Automobile.it URL: https://www.automobile.it/opel-corsa-30000?anno_fino_a=2025&immatricolazione=2020&prezzo_da=3000
Autosupermarket URL: https://autosupermarket.it/ricerca/opel/corsa?prezzo-da-3000&prezzo-a=30000&anno-da=2020&anno-a=2025
Subito.it URL: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?ps=3000&pe=30000&ys=2020&ye=2025
Autotorino URL: https://www.autotorino.it/veicoli/auto?immatricolazione_anno=2020-2025&marca_sito=818&modello_sito=1028&price=3000-30000
Autoscout URLs: ['https://www.autoscout24.it/lst/opel/corsa?atype=C&cy=I&damaged_listing=exclude&desc=1&fregfrom=2020&fregto=2025&powertype=kw&pricefrom=3000&priceto=4000&search_id=2bu1025n5if&sort=standard&source=homepage_search-mask&ustate=N%2CU', 'https://www.autoscout24.it/lst/opel/corsa?atype=C&cy=I&damaged_listing=exclude&desc=1&fregfrom=2020&fregto=2025&powertype=kw&pricefrom=4000&priceto=5000&search_id=2bu1025n5if&sort=standard&source=homepage_search-mask&ustate=N%2CU', 'https://www.autoscout24.it/lst/

### 2. Scraping - Autoscout

In [9]:
# Lista per raccogliere tutti i DataFrame
all_dataframes = []

# Ciclo su tutti gli URL
for i, url in enumerate(urls_as, start=1):
    print(f"\n🔍 Scraping URL {i}/{len(urls_as)}: {url}")
    try:
        df_temp = autoscout_scraper(url_template=url + "&page={}", max_pages=20)
        if df_temp is not None and not df_temp.empty:
            df_temp['Categoria'] = f'URL_{i}'  # colonna per sapere da quale URL proviene
            all_dataframes.append(df_temp)
        else:
            print(f"⚠ Nessun dato trovato per URL {i}")
    except Exception as e:
        print(f"❌ Errore durante lo scraping dell'URL {i}: {e}")
        continue  # passa al prossimo URL anche se c'è un errore

# Concatenazione dei tutti i DataFrame in uno solo
if all_dataframes:
    df_as = pd.concat(all_dataframes, ignore_index=True)
    print("\n✅ Raccolti tutti i dati!")
    print(df_as.head())
else:
    print("❌ Nessun dato trovato da nessun URL.")


🔍 Scraping URL 1/27: https://www.autoscout24.it/lst/opel/corsa?atype=C&cy=I&damaged_listing=exclude&desc=1&fregfrom=2020&fregto=2025&powertype=kw&pricefrom=3000&priceto=4000&search_id=2bu1025n5if&sort=standard&source=homepage_search-mask&ustate=N%2CU
Scraping page 1...
Nessun annuncio trovato nella pagina 1. Terminando.
❌ Errore durante lo scraping dell'URL 1: 'Prezzo'

🔍 Scraping URL 2/27: https://www.autoscout24.it/lst/opel/corsa?atype=C&cy=I&damaged_listing=exclude&desc=1&fregfrom=2020&fregto=2025&powertype=kw&pricefrom=4000&priceto=5000&search_id=2bu1025n5if&sort=standard&source=homepage_search-mask&ustate=N%2CU
Scraping page 1...
Scraping page 2...
Nessun annuncio trovato nella pagina 2. Terminando.

🔍 Scraping URL 3/27: https://www.autoscout24.it/lst/opel/corsa?atype=C&cy=I&damaged_listing=exclude&desc=1&fregfrom=2020&fregto=2025&powertype=kw&pricefrom=5000&priceto=6000&search_id=2bu1025n5if&sort=standard&source=homepage_search-mask&ustate=N%2CU
Scraping page 1...
Scraping page 

In [ ]:
"""
# Lista per raccogliere tutti i DataFrame
all_dataframes = []

# Ciclo su tutti gli URL
for i, url in enumerate(urls_as, start=1):
    print(f"\n🔍 Scraping URL {i}/{len(urls_as)}...")
    df_temp = autoscout_scraper(url_template=url + "&page={}", max_pages=20)
    if df_temp is not None and not df_temp.empty:
        df_temp['Categoria'] = f'URL_{i}'  #colonna per sapere da quale URL proviene
        all_dataframes.append(df_temp)
    else:
        print(f"⚠ Nessun dato trovato per URL {i}")

# Concatenazione dei tutti i DataFrame in uno solo
if all_dataframes:
    df_as = pd.concat(all_dataframes, ignore_index=True)
    print("\n✅ Raccolti tutti i dati!")
    print(df_as.head())
else:
    print("❌ Nessun dato trovato da nessun URL.")
    
"""

In [10]:
print(df_as.info())
df_as.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Annuncio           2469 non-null   object 
 1   Link               2469 non-null   object 
 2   Marca              2469 non-null   object 
 3   Modello            2469 non-null   object 
 4   Modello_plus_info  2459 non-null   object 
 5   Prezzo             2386 non-null   float64
 6   Chilometraggio     2469 non-null   int64  
 7   Cambio             2469 non-null   object 
 8   Immatricolazione   2469 non-null   object 
 9   Carburante         2469 non-null   object 
 10  CV                 2469 non-null   object 
 11  Località           2469 non-null   object 
 12  Venditore          2469 non-null   object 
 13  Categoria          2469 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 270.2+ KB
None


,Annuncio,Link,Marca,Modello,Modello_plus_info,Prezzo,Chilometraggio,Cambio,Immatricolazione,Carburante,CV,Località,Venditore,Categoria
0,Opel Corsa 1.5 Diesel Start/Stop Elegance PREZ...,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,1.5 Diesel Start/Stop Elegance PREZZO +IVA,4111.0,150000,Manuale,01/2021,Diesel,75 kW (102 CV),Elena • IT-38017 Mezzolombardo - Trento,Rivenditore,URL_2
1,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,Corsa 1.2 Blitz Edition s,6200.0,6000,Manuale,03/2022,Benzina,55 kW (75 CV),Steven Mazzucco • IT-36016 Thiene,Rivenditore,URL_3
2,Opel Corsa 1.5 GS Line s&s 100cv,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,1.5 GS Line s&s 100cv,7990.0,200000,Manuale,11/2020,Diesel,74 kW (101 CV),"Privato,IT-81039 villa literno",Privato,URL_4
3,Opel Corsa Corsa 1.2 Blitz Edition s,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,Corsa 1.2 Blitz Edition s,6200.0,6000,Manuale,03/2022,Benzina,55 kW (75 CV),Steven Mazzucco • IT-36016 Thiene,Rivenditore,URL_4
4,Opel Corsa 1.4 Turbo Start/Stop On,https://www.autoscout24.it/annunci/opel-corsa-...,Opel,Corsa,1.4 Turbo Start/Stop On,7990.0,146000,Manuale,07/2025,Benzina,74 kW (101 CV),"Privato,IT-06124 Perugia",Privato,URL_4


### 3. Scraping - Automobili.it

In [ ]:
df_at = automobile_it_scraper(url_at)

In [ ]:
df_at

In [ ]:
df_at.info()

### 4. Scraping - Autosupermarket

In [ ]:
df_asm = autosupermarket_scraper(url_asm)

In [ ]:
df_asm['Prezzo'] = df_asm['Prezzo'].apply(lambda x: x.split('\n')[-1] if '%' in x else x)

In [ ]:
df_asm

In [ ]:
for col in df_asm.columns:
    #print(f"{col}: {df_asm[col].isnull().sum()} valori mancanti")
    #print(f"{col}: {df_asm[col].unique()} valori unici")
    print(f"- {col}: {(df_asm[col] == '').sum()} somma dei valori vuoti")

In [ ]:
df_asm.replace('', np.nan, inplace=True)
df_asm.replace('Prezzo riservato', np.nan, inplace=True)

In [ ]:
df_asm.info()

In [ ]:
df_asm.dropna(inplace=True)    

### 5. Scraping - Subito.it

In [12]:
df_su = subito_scraper(url_su)


Caricamento pagina 1: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?ps=3000&ys=2020&ye=2025
✅ Cookie accettati
✅ Trovati 33 annunci nella pagina 1

Caricamento pagina 2: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=2&ps=3000&ys=2020&ye=2025
✅ Trovati 33 annunci nella pagina 2

Caricamento pagina 3: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=3&ps=3000&ys=2020&ye=2025
✅ Trovati 33 annunci nella pagina 3

Caricamento pagina 4: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=4&ps=3000&ys=2020&ye=2025
✅ Trovati 33 annunci nella pagina 4

Caricamento pagina 5: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=5&ps=3000&ys=2020&ye=2025
✅ Trovati 33 annunci nella pagina 5

Caricamento pagina 6: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=6&ps=3000&ys=2020&ye=2025
✅ Trovati 33 annunci nella pagina 6

Caricamento pagina 7: https://www.subito.it/annunci-italia/vendita/auto/opel/corsa/?o=7&ps

In [13]:
for col in df_su.columns:
    print(f"- {col}: {(df_su[col] == '').sum()} somma dei valori vuoti")

- Annuncio: 0 somma dei valori vuoti
- Prezzo: 0 somma dei valori vuoti
- Tipologia: 0 somma dei valori vuoti
- Immatricolazione: 0 somma dei valori vuoti
- Chilometraggio: 0 somma dei valori vuoti
- Carburante: 0 somma dei valori vuoti
- Cambio: 0 somma dei valori vuoti
- Classe Euro: 0 somma dei valori vuoti
- Località: 0 somma dei valori vuoti
- Provincia: 0 somma dei valori vuoti
- Rivenditore: 0 somma dei valori vuoti
- Link: 0 somma dei valori vuoti


In [14]:
df_su.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Annuncio          2069 non-null   object
 1   Prezzo            2069 non-null   object
 2   Tipologia         2069 non-null   object
 3   Immatricolazione  2069 non-null   object
 4   Chilometraggio    1901 non-null   object
 5   Carburante        2062 non-null   object
 6   Cambio            2037 non-null   object
 7   Classe Euro       1760 non-null   object
 8   Località          2069 non-null   object
 9   Provincia         2069 non-null   object
 10  Rivenditore       2069 non-null   object
 11  Link              2069 non-null   object
dtypes: object(12)
memory usage: 194.1+ KB


In [15]:
df_su

,Annuncio,Prezzo,Tipologia,Immatricolazione,Chilometraggio,Carburante,Cambio,Classe Euro,Località,Provincia,Rivenditore,Link
0,Opel Corsa 1.2 Elegance OK NEOPATENTATI,9.400 €,Usato,12/2021,135000 Km,Benzina,Manuale,Euro 6,Pistoia,PT,Privato,https://www.subito.it/auto/opel-corsa-1-2-eleg...
1,OPEL Corsa 6ª serie - Corsa 1.5 D 100 CV Edition,14.800 €,Usato,12/2022,57171 Km,Diesel,Manuale,Euro 6,Villafranca di Verona,VR,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...
2,Opel Corsa VI 2023 1.2 GS s&s 75cv,13.500 €,Usato,03/2024,22658 Km,Benzina,Manuale,Euro 6,Erba,CO,Privato,https://www.subito.it/auto/opel-corsa-vi-2023-...
3,Opel corsa 1.2,14.800 €,Usato,03/2024,12000 Km,Benzina,Manuale,Euro 6,Viterbo,VT,Privato,https://www.subito.it/auto/opel-corsa-1-2-vite...
4,Opel Corsa 6 Serie Corsa 1.2 Edition,13.900 €,Usato,06/2021,41338 Km,Benzina,Manuale,Euro 6,San Fior,TV,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,Opel Corsa 6 Serie 1.2,12.000 €,Usato,09/2020,90000 Km,Benzina,Manuale,Euro 6,Brescia,BS,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...
2065,Opel Corsa 6 Serie 2023,12.300 €,Usato,05/2023,36219 Km,Benzina,Manuale,None,Casalnuovo di Napoli,NA,Privato,https://www.subito.it/auto/opel-corsa-6-serie-...
2066,Opel Corsa F 2022,13.600 €,Usato,02/2022,39000 Km,Benzina,Manuale,Euro 6,Guidonia Montecelio,RM,Privato,https://www.subito.it/auto/opel-corsa-f-2022-r...
2067,Opel Corsa 1.2 100CV Elegance,14.982 €,Usato,06/2022,31679 Km,Benzina,Manuale,Euro 6,Meda,MB,Privato,https://www.subito.it/auto/opel-corsa-1-2-100c...


### 7. Export dei dati

In [5]:
project_root = Path.cwd().parents[1]

# Percorsi delle cartelle contenenti i moduli
output_folder = project_root / 'Data' / 'Raw_data' / modello_per_analisi
# Aggiungi al sys.path
sys.path.extend(str(output_folder))
#Verifica se la cartella esiste, se no la crea
output_folder.mkdir(parents=True, exist_ok=True)

In [12]:
def salva_df(df, output_path, modello):
    if df is not None and not df.empty:
        df.to_csv(output_path, index=False)
        print(f"""💾 File salvato, del modello {modello},
    nel percorso: {output_path}""")
    else:
        print(f"⚠️ Nessun dato da salvare per il modello {modello} in {output_path}.")

In [13]:
try:
    salva_df(df_as.drop(columns='Categoria', errors='ignore'), output_folder / "data_autoscout.csv", modello_per_analisi)
except Exception as e:
    print(f"⚠️ Errore con df_as: {e}")

try:
    salva_df(df_at, output_folder / "data_automobile_it.csv", modello_per_analisi)
except Exception as e:
    print(f"⚠️ Errore con df_at: {e}")

try:
    salva_df(df_asm, output_folder / "data_autosupermarket.csv", modello_per_analisi)
except Exception as e:
    print(f"⚠️ Errore con df_asm: {e}")

try:
    salva_df(df_su, output_folder / "data_subito_it.csv", modello_per_analisi)
except Exception as e:
    print(f"⚠️ Errore con df_su: {e}")

try:
    salva_df(df_ar, output_folder / "data_autotorino.csv", modello_per_analisi)
except Exception as e:
    print(f"⚠️ Errore con df_ar: {e}")


💾 File salvato, del modello Opel_Corsa,
    nel percorso: /Users/federicogamberini/VS Code/Vehicle_Price_Monitor/Materiali/Dataset_grezzi/Opel_Corsa/data_autoscout.csv
⚠️ Errore con df_at: name 'df_at' is not defined
⚠️ Errore con df_asm: name 'df_asm' is not defined
⚠️ Errore con df_su: name 'df_su' is not defined
⚠️ Errore con df_ar: name 'df_ar' is not defined


In [14]:
'''
# Salvataggio del file
output_file_as = output_folder / "data_autoscout.csv"
df_as = df_as.drop(columns='Categoria', errors='ignore')
df_as.to_csv(output_file_as, index=False)
print(f"""💾 File salvato, del modello {modello_per_analisi},
   nel percorso: {output_file_as}""")

output_file_at = output_folder / "data_automobile_it.csv"
df_at.to_csv(output_file_at, index=False)
print(f"""💾 File salvato, del modello {modello_per_analisi},
   nel percorso: {output_file_at}""")

output_file_asm = output_folder / "data_autosupermarket.csv"
df_asm.to_csv(output_file_asm, index=False)
print(f"""💾 File salvato, del modello {modello_per_analisi},
   nel percorso: {output_file_asm}""")

output_file_su = output_folder / "data_subito_it.csv"
df_su.to_csv(output_file_su, index=False)
print(f"""💾 File salvato, del modello {output_file_su},
   nel percorso: {output_file_su}""")

output_file_au = output_folder / "data_autotorino.csv"
df_au.to_csv(output_file_au, index=False)
print(f"""💾 File salvato, del modello {output_file_au},
   nel percorso: {output_file_au}""")

'''

'\n# Salvataggio del file\noutput_file_as = output_folder / "data_autoscout.csv"\ndf_as = df_as.drop(columns=\'Categoria\', errors=\'ignore\')\ndf_as.to_csv(output_file_as, index=False)\nprint(f"""💾 File salvato, del modello {modello_per_analisi},\n   nel percorso: {output_file_as}""")\n\noutput_file_at = output_folder / "data_automobile_it.csv"\ndf_at.to_csv(output_file_at, index=False)\nprint(f"""💾 File salvato, del modello {modello_per_analisi},\n   nel percorso: {output_file_at}""")\n\noutput_file_asm = output_folder / "data_autosupermarket.csv"\ndf_asm.to_csv(output_file_asm, index=False)\nprint(f"""💾 File salvato, del modello {modello_per_analisi},\n   nel percorso: {output_file_asm}""")\n\noutput_file_su = output_folder / "data_subito_it.csv"\ndf_su.to_csv(output_file_su, index=False)\nprint(f"""💾 File salvato, del modello {output_file_su},\n   nel percorso: {output_file_su}""")\n\noutput_file_au = output_folder / "data_autotorino.csv"\ndf_au.to_csv(output_file_au, index=False)\